In [155]:
# Imports
import requests
from bs4 import BeautifulSoup
import psycopg2

In [156]:
# Variablen
conn_Ok = True


In [157]:
# Check if connection to DB possible
try:
    connection = psycopg2.connect(
        dbname='PSQL_ADSFS2023Gruppe15',
        user='ADSFS2023Gruppe15',
        password='ADS_FS_2023_G15!?',
        host='localhost',
        port='5432'
    )

    cursor = connection.cursor()

except (Exception, psycopg2.Error) as error:
        print('Fehler beim Verbinden mit der PostgreSQL-Datenbank:', error)
        conn_Ok = False

In [158]:
def get_team_id(team_name):

    if team_name == 'Bor. Mönchengladbach':
        team_name = 'Borussia Mönchengladbach'
    elif team_name == '1899 Hoffenheim':
        team_name = 'TSG 1899 Hoffenheim'
    elif team_name == 'SV Werder Bremen':
        team_name = 'Werder Bremen'
        
    # Vergleicht Team Namen und sucht nach ID
    cursor.execute("SELECT team_id FROM bundesliga_mannschaften WHERE mannschaft = %s", (team_name,))
    result = cursor.fetchone()[0]

    if result:
        return result
    else:
        print(f'Keine Übereinstimmung für {team_name} gefunden')


In [160]:
if conn_Ok:
    # Anzahl Pässe für die spielTage im angegebenen Range
    for matchday in range(1, 35):
        url = f'https://www.sport.de/fussball/deutschland-bundesliga/se45495/2022-2023/ro132754/spieltag/md{matchday}/teamstatistik-paesse/'

        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Suche nach allen Mannschaftsnamen und Passquoten
            team_elements = soup.find_all('td', class_='team-name team-name-list')
            pass_complete_elements = soup.find_all('td', class_='team_stats-passes_complete team_stats-passes_complete-list')
            pass_failed_elements = soup.find_all('td', class_='team_stats-passes_failed team_stats-passes_failed-list')
            pass_total_elements = soup.find_all('td', class_='team_stats-passes team_stats-passes-list')
            pass_percentage_elements = soup.find_all('td', class_='team_stats-passes_complete_percentage team_stats-passes_complete_percentage-format')

            # Iteriere über die gefundenen Elemente und extrahiere die gewünschten Informationen
            for team_element,pass_complete_element, pass_failed_element,pass_total_element, pass_percentage_element in zip(team_elements,pass_complete_elements, pass_failed_elements, pass_total_elements, pass_percentage_elements):
                team_name = team_element.text.strip()
                pass_complete = pass_complete_element.text.strip()
                pass_failed = pass_failed_element.text.strip()
                pass_total = pass_total_element.text.strip()
                pass_percentage = pass_percentage_element.text.strip()

                # Speichere die Daten in der PostgreSQL-Datenbank
                insert_data_query = '''
                INSERT INTO bundesliga_pass_stats (matchday, team_id, pass_complete, pass_failed, pass_total, pass_percentage) 
                VALUES (%s, %s, %s,%s, %s, %s);
                '''

                team_id = get_team_id(team_name)

                cursor.execute(insert_data_query, (matchday, team_id, pass_complete, pass_failed, pass_total, float(pass_percentage.replace(',', '.'))))

                connection.commit()
        else:
            print(f'Fehler beim Abrufen der Webseite für Spieltag {matchday}')

65 1. FC Köln
87 Borussia Mönchengladbach
40 FC Bayern München
1635 RB Leipzig
6 Bayer Leverkusen
129 VfL Bochum
95 FC Augsburg
131 VfL Wolfsburg
7 Borussia Dortmund
54 Hertha BSC
175 TSG 1899 Hoffenheim
80 1. FC Union Berlin
91 Eintracht Frankfurt
112 SC Freiburg
16 VfB Stuttgart
134 Werder Bremen
81 1. FSV Mainz 05
9 FC Schalke 04
40 FC Bayern München
87 Borussia Mönchengladbach
134 Werder Bremen
1635 RB Leipzig
175 TSG 1899 Hoffenheim
7 Borussia Dortmund
91 Eintracht Frankfurt
65 1. FC Köln
6 Bayer Leverkusen
16 VfB Stuttgart
9 FC Schalke 04
129 VfL Bochum
81 1. FSV Mainz 05
80 1. FC Union Berlin
54 Hertha BSC
112 SC Freiburg
131 VfL Wolfsburg
95 FC Augsburg
1635 RB Leipzig
131 VfL Wolfsburg
16 VfB Stuttgart
40 FC Bayern München
87 Borussia Mönchengladbach
6 Bayer Leverkusen
112 SC Freiburg
175 TSG 1899 Hoffenheim
7 Borussia Dortmund
81 1. FSV Mainz 05
134 Werder Bremen
54 Hertha BSC
129 VfL Bochum
91 Eintracht Frankfurt
95 FC Augsburg
80 1. FC Union Berlin
9 FC Schalke 04
65 1. FC 

In [161]:
for matchday in range(1, 35):
    url = f'https://www.sport.de/fussball/deutschland-bundesliga/se45495/2022-2023/ro132754/spieltag/md{matchday}/teamstatistik-torschuesse/'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        team_elements = soup.find_all('td', class_='team-name team-name-list')
        shots_elements = soup.find_all('td', class_='team_stats-shots team_stats-shots-list')

        for team_element, shots_element in zip(team_elements, shots_elements):
            team_name = team_element.text.strip()
            shots_total = shots_element.text.strip()

            insert_data_query = '''
                INSERT INTO bundesliga_shots_stats (matchday, team_id, shots_total)
                VALUES (%s, %s, %s);
            '''
            team_id = get_team_id(team_name)

            cursor.execute(insert_data_query, (matchday, team_id, shots_total))

            connection.commit()

    else:
        print(f'Fehler beim Abrufen der Webseite für Spieltag {matchday} und Torschüsse')

175 TSG 1899 Hoffenheim
40 FC Bayern München
112 SC Freiburg
134 Werder Bremen
9 FC Schalke 04
80 1. FC Union Berlin
87 Borussia Mönchengladbach
81 1. FSV Mainz 05
131 VfL Wolfsburg
91 Eintracht Frankfurt
54 Hertha BSC
65 1. FC Köln
16 VfB Stuttgart
7 Borussia Dortmund
129 VfL Bochum
1635 RB Leipzig
6 Bayer Leverkusen
95 FC Augsburg
134 Werder Bremen
95 FC Augsburg
1635 RB Leipzig
7 Borussia Dortmund
129 VfL Bochum
87 Borussia Mönchengladbach
9 FC Schalke 04
16 VfB Stuttgart
175 TSG 1899 Hoffenheim
91 Eintracht Frankfurt
54 Hertha BSC
40 FC Bayern München
65 1. FC Köln
112 SC Freiburg
6 Bayer Leverkusen
131 VfL Wolfsburg
80 1. FC Union Berlin
81 1. FSV Mainz 05
7 Borussia Dortmund
175 TSG 1899 Hoffenheim
80 1. FC Union Berlin
40 FC Bayern München
134 Werder Bremen
65 1. FC Köln
95 FC Augsburg
81 1. FSV Mainz 05
112 SC Freiburg
91 Eintracht Frankfurt
87 Borussia Mönchengladbach
1635 RB Leipzig
9 FC Schalke 04
129 VfL Bochum
6 Bayer Leverkusen
131 VfL Wolfsburg
54 Hertha BSC
16 VfB Stutt

In [164]:
for matchday in range(1, 35):
    url = f'https://www.sport.de/fussball/deutschland-bundesliga/se45495/2022-2023/ro132754/spieltag/md{matchday}/teamstatistik-zweikaempfe/'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        team_elements = soup.find_all('td', class_='team-name team-name-list')
        duels_elements = soup.find_all('td', class_='team_stats-duels team_stats-duels-list')
        duels_won_elements = soup.find_all('td', class_='team_stats-duels_won team_stats-duels_won-list')

        for team_element, duels_element, duels_won_element in zip(team_elements, duels_elements, duels_won_elements):
            team_name = team_element.text.strip()
            duels_total = duels_element.text.strip()
            duels_won = duels_won_element.text.strip()

            insert_data_query = '''
                INSERT INTO bundesliga_duels_stats (matchday, team_id, duels_total, duels_won)
                VALUES (%s, %s, %s, %s);
                '''
            team_id =get_team_id(team_name)
        
            cursor.execute(insert_data_query, (matchday, team_id, duels_total, duels_won))
                        
            connection.commit()

    else:
        print(f'Fehler beim Abrufen der Webseite für Spieltag {matchday} und Zweikämpfe')

112 SC Freiburg
65 1. FC Köln
40 FC Bayern München
175 TSG 1899 Hoffenheim
7 Borussia Dortmund
131 VfL Wolfsburg
1635 RB Leipzig
81 1. FSV Mainz 05
80 1. FC Union Berlin
54 Hertha BSC
129 VfL Bochum
16 VfB Stuttgart
134 Werder Bremen
6 Bayer Leverkusen
87 Borussia Mönchengladbach
91 Eintracht Frankfurt
9 FC Schalke 04
95 FC Augsburg
40 FC Bayern München
6 Bayer Leverkusen
87 Borussia Mönchengladbach
81 1. FSV Mainz 05
7 Borussia Dortmund
1635 RB Leipzig
91 Eintracht Frankfurt
175 TSG 1899 Hoffenheim
16 VfB Stuttgart
134 Werder Bremen
129 VfL Bochum
54 Hertha BSC
65 1. FC Köln
112 SC Freiburg
80 1. FC Union Berlin
9 FC Schalke 04
95 FC Augsburg
131 VfL Wolfsburg
81 1. FSV Mainz 05
40 FC Bayern München
16 VfB Stuttgart
1635 RB Leipzig
87 Borussia Mönchengladbach
7 Borussia Dortmund
65 1. FC Köln
6 Bayer Leverkusen
9 FC Schalke 04
131 VfL Wolfsburg
175 TSG 1899 Hoffenheim
91 Eintracht Frankfurt
134 Werder Bremen
54 Hertha BSC
80 1. FC Union Berlin
112 SC Freiburg
129 VfL Bochum
95 FC Augs